In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import copy
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut, KFold
from sklearn.svm import SVR
warnings.filterwarnings('ignore')
%matplotlib inline

In [35]:
def rename(name):
    name = str(name)
    if name[0] == '_':
        name = name[1:]
    return name

In [37]:
path_data = pd.read_excel('../Data/18 drugs data/Cell_lines_for_18_drugs_glio.xlsx')
path_data.index = path_data['row.names']
path_data = path_data.drop(['row.names', 'Score'], axis=1)
print(path_data.shape[0], 'cell lines')
print(path_data.shape[1], 'paths')
path_data.index = list(map(rename, path_data.index))
path_data.head()

227 cell lines
29 paths


,AKT_Pathway,Androgen_receptor_Pathway,ATM_Pathway,ATM_Pathway_Apoptosis,ATM_Pathway_Cell_Survival,cAMP_Pathway,Caspase_Cascade,Cellular_Anti_Apoptosis_Pathway,CREB_Pathway,DNA_damage_induced_apoptosis_effect,...,MAPK_Signaling_Pathway,Mitochondrial_Apopotosis_m_Pathway,p38_m_Signaling_Pathway,p38_p_Signaling_Pathway,p53_Signaling_m_Pathway,p53_Signaling_m_Pathway_p53_Degradation,PAK_Pathway,PAK_Pathway_Actin_Cytoskeleton,PPAR_Pathway,RAS_Pathway
697,44.732345,-17.407952,9.609132,1.169696,1.185616,9.888184,-8.456224,8.724404,-40.245975,27.073601,...,32.576476,-14.602021,71.221712,71.221712,13.481800,22.313834,8.751191,29.635535,18.377713,62.659360
8_MG_BA,71.843846,-0.370608,14.855253,2.874575,3.851498,38.057946,-9.970357,22.871026,-39.163241,22.188627,...,89.378684,-9.178359,102.534602,103.686684,9.655002,22.144591,58.956036,50.961608,19.925612,18.796660
A101D,65.659676,14.497817,4.296379,2.095332,-1.445586,54.386958,-11.962294,29.767217,-32.455694,7.317787,...,122.211813,-42.684066,88.982284,90.376803,5.185066,21.189273,67.778562,54.587248,32.266822,22.110341
A253,103.723684,8.659669,2.867404,1.095290,2.230086,75.755789,-52.967854,38.395007,-9.304220,10.818151,...,136.919206,-35.072105,142.385682,142.385682,34.385299,39.407664,53.822328,62.855762,32.341330,44.767662
A3_KAW,72.914616,-21.321219,15.074902,0.472201,8.137782,34.408836,23.717182,7.714501,-55.155752,15.112377,...,34.552454,-21.489897,115.625484,115.625484,24.733140,33.424749,41.656193,55.700606,22.729198,15.003385


In [38]:
def rename(name):
    return str(name).replace('-','_')

In [39]:
ic50_data = pd.read_excel('../Data/18 drugs data/cancerrxgene_ic50.xlsx')
ic50_data.index = ic50_data['ic50']
ic50_data = ic50_data.drop('ic50', axis=1)
ic50_data.index = list(map(rename, ic50_data.index))
print(ic50_data.shape[0], 'cell lines')
print(ic50_data.shape[1], 'drugs')
ic50_data.head()

227 cell lines
22 drugs


,Axitinib_IC_50,Bexarotene_IC_50,Bicalutamide_IC_50,Bortezomib_IC_50,Bosutinib_IC_50,Dasatinib_IC_50,Docetaxel_IC_50,Erlotinib_IC_50,Etoposide_IC_50,Gefitinib_IC_50,...,Lapatinib_IC_50,Lenalidomide_IC_50,Methotrexate_IC_50,Nilotinib_IC_50,Paclitaxel_IC_50,Pazopanib_IC_50,Sorafenib_IC_50,Sunitinib_IC_50,Temsirolimus_IC_50,Vorinostat_IC_50
697,0.858777,5.389585,5.315078,-6.220779,-1.588857,-1.610100,-5.822296,0.975711,-1.413875,4.154314,...,4.884310,4.792804,-4.027764,1.119575,-6.505565,3.092866,1.954430,-0.487360,-4.535688,-0.902847
8_MG_BA,1.450813,2.614488,4.497036,-8.294864,1.675750,-1.867058,-7.207004,4.377018,-3.060272,1.823547,...,4.651814,5.988425,3.082169,4.440559,-4.833526,3.582369,4.917932,1.874118,-0.895439,0.849750
A101D,1.599112,5.703109,4.687344,-8.397833,1.032470,0.029973,-5.156745,3.357381,0.180757,1.428544,...,3.037769,4.006487,1.244383,1.936639,-4.646205,4.513274,3.062231,1.023543,-2.627070,0.108349
A253,3.697556,5.680654,4.465783,-8.475787,2.653229,1.317068,-4.673750,1.012181,-3.993972,2.741325,...,0.679735,6.236156,1.322785,5.456089,-4.052458,2.432830,3.533092,5.618408,-0.870013,1.075108
A3_KAW,4.721577,4.707440,5.297175,-6.658239,4.095641,-0.015604,0.233008,1.693679,-1.026487,3.372308,...,5.533987,5.915477,-3.231484,5.028748,-2.610322,3.570335,0.633504,0.485789,-1.607818,-0.753816


In [40]:
def rename(name):
    return str(name)[7:].replace('-','_')

In [41]:
gene_data = pd.read_table('../Data/18 drugs data/CRxG.txt')
gene_names = gene_data['SYMBOL']
gene_data = gene_data.drop('SYMBOL', axis=1)
gene_data = gene_data.T
gene_data.columns = gene_names
print(gene_data.shape[0], 'cell lines')
print(gene_data.shape[1], 'features')
gene_data.index = list(map(rename, gene_data.index))
gene_data.head()

251 cell lines
12045 features


SYMBOL,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAGAB,AAK1,AAMDC,...,ZSWIM8-AS1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
697,4.438244,4.438244,6.530357,4.438244,179.964275,57.806721,4.438244,47.217428,4.581935,99.931543,...,30.534343,360.075293,609.973423,4959.331983,4.851226,4.438244,188.580219,40.560550,8.529712,791.122462
8_MG_BA,4.438244,4.438244,6.530357,4.438244,195.244161,221.594233,4.438244,77.839352,4.438244,92.811499,...,9.074232,234.035644,247.170079,2093.184558,4.851226,4.438244,129.354628,384.020608,7.892836,1372.471460
A101D,4.438244,717.328383,5.346804,4.438244,390.581735,728.301303,4.438244,73.461249,4.438244,78.593417,...,9.074232,268.987959,23.503397,232.340345,4.851226,4.438244,233.558718,437.416189,8.413118,342.663367
A253,4.438244,4.438244,11.165160,4.438244,69.139244,355.916910,4.438244,75.824934,4.438244,56.745750,...,9.169521,335.910881,257.675793,638.620473,4.904587,4.438244,95.812516,72.947420,9.149054,531.706244
A3_KAW,5.598161,6.016490,6.530357,5.263819,307.833642,817.854683,20.986101,73.461249,4.438244,75.540123,...,13.788840,381.084450,51.147302,792.083722,6.859692,6.304038,203.302547,167.520136,10.212146,453.056142


In [42]:
gene_data = pd.concat([gene_data, ic50_data['Axitinib_IC_50']], axis=1, join='inner')
gene_data = gene_data.drop('Axitinib_IC_50', axis=1)
print(gene_data.shape[0], 'cell lines')
print(gene_data.shape[1], 'features')
gene_data.head()

227 cell lines
12045 features


,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAGAB,AAK1,AAMDC,...,ZSWIM8-AS1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
697,4.438244,4.438244,6.530357,4.438244,179.964275,57.806721,4.438244,47.217428,4.581935,99.931543,...,30.534343,360.075293,609.973423,4959.331983,4.851226,4.438244,188.580219,40.560550,8.529712,791.122462
8_MG_BA,4.438244,4.438244,6.530357,4.438244,195.244161,221.594233,4.438244,77.839352,4.438244,92.811499,...,9.074232,234.035644,247.170079,2093.184558,4.851226,4.438244,129.354628,384.020608,7.892836,1372.471460
A101D,4.438244,717.328383,5.346804,4.438244,390.581735,728.301303,4.438244,73.461249,4.438244,78.593417,...,9.074232,268.987959,23.503397,232.340345,4.851226,4.438244,233.558718,437.416189,8.413118,342.663367
A253,4.438244,4.438244,11.165160,4.438244,69.139244,355.916910,4.438244,75.824934,4.438244,56.745750,...,9.169521,335.910881,257.675793,638.620473,4.904587,4.438244,95.812516,72.947420,9.149054,531.706244
A3_KAW,5.598161,6.016490,6.530357,5.263819,307.833642,817.854683,20.986101,73.461249,4.438244,75.540123,...,13.788840,381.084450,51.147302,792.083722,6.859692,6.304038,203.302547,167.520136,10.212146,453.056142


In [44]:
# data saving
gene_data.to_csv('../Data/18 drugs data/gene data.csv')
path_data.to_csv('../Data/18 drugs data/path data.csv')
ic50_data.to_csv('../Data/18 drugs data/ic50 data.csv')